<a href="https://colab.research.google.com/github/amina04/CNN-LSTM_algo/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello rafika')

hello rafika
